# Crawling PTA & Berita

In [1]:
!pip install builtwith

  Preparing metadata (setup.py) ... done
  Created wheel for builtwith: filename=builtwith-1.3.4-py3-none-any.whl size=36077 sha256=ff473db96112300b080d9592c579347bbbcb3135d6efb9786068f130ebe172ba
  Stored in directory: /root/.cache/pip/wheels/7f/2d/b2/606e3df914d4aeeab99c4a4e3e9a61673d2293c2e346db00c8
Successfully built builtwith


## 1. Crawling PTA

In [ ]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://pta.trunojoyo.ac.id')
print(res)

{'web-servers': ['Nginx'], 'javascript-frameworks': ['jQuery', 'jQuery UI']}


## Page & Link Keluar PTA

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua judul h1, h2, h3
        headings = soup.find_all(['h1', 'h2', 'h3'])
        for heading in headings:
            print(f"{heading.name}: {heading.get_text()}")

        # Ambil semua link
        links = soup.find_all('a', href=True)
        for link in links:
            print(f"URL: {link['href']} | Teks: {link.get_text()}")

    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan saat mengakses {url}: {e}")

# Gunakan fungsi
crawl_website("https://pta.trunojoyo.ac.id/")

h2: Daftar Karya Ilmiah
URL: index.html | Teks: 
URL: # | Teks: 14677Journal
URL: https://pta.trunojoyo.ac.id/ | Teks: Beranda
URL: https://pta.trunojoyo.ac.id/c_search/ | Teks: Pencarian
URL: https://pta.trunojoyo.ac.id/c_template/ | Teks: Download
URL: https://library.trunojoyo.ac.id/detil.php?id=23 | Teks: Petunjuk Upload
URL: https://pta.trunojoyo.ac.id/c_contact/ | Teks: Kontak
URL: # | Teks: STRATEGI PENGEMBANGAN MAKANAN DAN MINUMAN KHAS PULAU GILIGENTING GUNA MENDUKUNG PARIWISATA BERKELANJUTAN
URL: https://pta.trunojoyo.ac.id/welcome/detail/170361100003 | Teks: Selengkapnya
URL: # | Teks: PERUMUSAN SANKSI PIDANA BAGI MASYARAKAT SEKITAR HUTAN YANG MELAKUKAN PENCURIAN KAYU MILIK NEGARA DALAM UNDANG-UNDANG NOMOR 18 TAHUN 2013
URL: https://pta.trunojoyo.ac.id/welcome/detail/170111100053 | Teks: Selengkapnya
URL: # | Teks: Peran Teor Motivasi Herzberg Sebagai Mediator Self Efficacy, Lingkungan Kerja Dalam Meningkatkan Prestasi Kerja Pegawai ( Kantor Jasa Penilai Publik Guntur Eki And

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

def get_text_or_na(soup, selectors):
    """Mencoba beberapa selektor untuk menemukan teks, mengembalikan 'N/A' jika tidak ditemukan."""
    for selector in selectors:
        element = soup.select_one(selector)
        if element:
            text = element.get_text(strip=True)
            if text and text.strip():
                return text.strip()
    return 'N/A'

def get_data_from_span(soup, text_contains):
    """Mencari span yang berisi teks tertentu dan mengekstrak nama setelah titik dua."""
    span = soup.select_one(f'span:-soup-contains("{text_contains}")')
    if span:
        text = span.get_text(strip=True)
        if ':' in text:
            return text.split(':', 1)[1].strip()
    return 'N/A'

def get_abstract_robust(soup, keywords):
    """
    Mencari tag heading (b) yang teksnya mengandung salah satu dari kata kunci
    dan mengekstrak paragraf berikutnya.
    """
    for tag_b in soup.find_all('b'):
        tag_text = tag_b.get_text(strip=True)
        if any(keyword.lower() in tag_text.lower() for keyword in keywords):
            parent_div = tag_b.find_parent('div')
            if parent_div:
                next_div_sibling = parent_div.find_next_sibling('div')
                if next_div_sibling:
                    p_tag = next_div_sibling.find('p', align="justify")
                    if p_tag:
                        abstract_text = p_tag.get_text(strip=True)
                        if abstract_text:
                            return abstract_text
    return 'N/A'

def get_total_pages(soup):
    """
    Mengekstrak total jumlah halaman dari navigasi paginasi.
    """
    try:
        pagination = soup.select_one('ol.pagination')
        if pagination:
            last_page_li = pagination.select('li')[-1]
            last_page_link = last_page_li.select_one('a')
            if last_page_link and 'href' in last_page_link.attrs:
                url_path = last_page_link['href']
                return int(url_path.split('/')[-1])
    except (IndexError, ValueError, KeyError):
        pass
    return 1

def scrape_teknik_all_data():
    """
    Scrapes all data from all study programs within the Faculty of Engineering.
    """
    # Hardcoded list of Engineering study programs and their URLs based on inspection
    prodi_data = [
        {'name': 'Teknik Industri', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/9'},
        {'name': 'Teknik Informatika', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/10'},
        {'name': 'Manajemen Informatika', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/11'},
        {'name': 'Teknik Multimedia Dan Jaringan', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/19'},
        {'name': 'Mekatronika', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/20'},
        {'name': 'Teknik Elektro', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/23'},
        {'name': 'Sistem Informasi', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/31'},
        {'name': 'Teknik Mesin', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/32'},
        {'name': 'Teknik Mekatronika', 'url': 'https://pta.trunojoyo.ac.id/c_search/byprod/33'}
    ]

    all_scraped_data = {
        "penulis": [], "judul": [], "pembimbing_pertama": [], "pembimbing_kedua": [],
        "abstrak_indonesia": [], "abstrak_inggris": [], "prodi": []
    }

    total_data_count = 0

    print("--- MULAI PROSES SCRAPING FAKULTAS TEKNIK ---")

    for prodi in prodi_data:
        print(f"\nScraping data for program: {prodi['name']}")
        print("-" * 50)

        first_page_url = prodi['url']
        try:
            r = requests.get(first_page_url)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            total_pages = get_total_pages(soup)
            print(f"Found {total_pages} pages for {prodi['name']}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching first page for {prodi['name']}: {e}", file=sys.stderr)
            total_pages = 1

        for i in range(1, total_pages + 1):
            url = f"{prodi['url']}/{i}"

            try:
                r = requests.get(url)
                r.raise_for_status()
                soup = BeautifulSoup(r.content, "html.parser")
                jurnals = soup.select('li[data-cat="#luxury"]')

                if not jurnals:
                    print(f"No more journals found on page {i}. Stopping for this program.")
                    break

                for jurnal in jurnals:
                    jurnal_url = jurnal.select_one('a.gray.button')['href']

                    try:
                        response = requests.get(jurnal_url)
                        response.raise_for_status()
                        soup1 = BeautifulSoup(response.content, "html.parser")
                        isi = soup1.select_one('div#content_journal')

                        if isi:
                            judul = get_text_or_na(isi, ['a.title', 'b.title', 'h2.title'])
                            penulis = get_data_from_span(isi, "Penulis")
                            pembimbing_pertama = get_data_from_span(isi, "Dosen Pembimbing I")
                            pembimbing_kedua = get_data_from_span(isi, "Dosen Pembimbing II")
                            abstrak_indonesia = get_abstract_robust(isi, ["Abstraksi", "Abstrak"])
                            abstrak_inggris = get_abstract_robust(isi, ["Abstraction", "Abstract", "ABSTRACT"])

                            all_scraped_data["penulis"].append(penulis)
                            all_scraped_data["judul"].append(judul)
                            all_scraped_data["pembimbing_pertama"].append(pembimbing_pertama)
                            all_scraped_data["pembimbing_kedua"].append(pembimbing_kedua)
                            all_scraped_data["abstrak_indonesia"].append(abstrak_indonesia)
                            all_scraped_data["abstrak_inggris"].append(abstrak_inggris)
                            all_scraped_data["prodi"].append(prodi['name'])

                            total_data_count += 1

                            # Print all data for the current entry
                            print(f"\n--- Data #{total_data_count} ---")
                            print(f"Prodi: {prodi['name']}")
                            print(f"Penulis: {penulis}")
                            print(f"Judul: {judul}")
                            print(f"Pembimbing 1: {pembimbing_pertama}")
                            print(f"Pembimbing 2: {pembimbing_kedua}")
                            print(f"Abstrak Indonesia: {abstrak_indonesia}")
                            print(f"Abstrak Inggris: {abstrak_inggris}")

                    except Exception as e:
                        print(f"ERROR: An error occurred while processing URL: {jurnal_url} - {e}", file=sys.stderr)

                time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL {url}: {e}", file=sys.stderr)
                continue

    df = pd.DataFrame(all_scraped_data)
    df.to_csv("pta_teknik_complete.csv", index=False)
    print("\n---")
    print(f"Scraping finished. Total data scraped: {total_data_count}")
    print("Data saved to pta_teknik_complete.csv")
    print("---")
    return df

# Run the full scraping process
scrape_teknik_all_data()

--- MULAI PROSES SCRAPING FAKULTAS TEKNIK ---

Scraping data for program: Teknik Industri
--------------------------------------------------
Found 143 pages for Teknik Industri

--- Data #1 ---
Prodi: Teknik Industri
Penulis: Siliwangi Fitra Rachmawanto S.T.
Judul: OPTIMASI PEMILIHAN PORTOFOLIO SAHAM PERUSAHAAN SEKTOR TAMBANG DENGAN MENGGUNAKAN GOAL PROGRAMMING
Pembimbing 1: Heri Awalul Ilhamsah S.T., M.T.
Pembimbing 2: Retno Indriartiningtias S.T., M.T.
Abstrak Indonesia: Portofolio adalah sekumpulan saham yang dimiliki investor. Dalam setiap saham pada portofolio memiliki resiko yang harus diminimalkan dengan cara diversifikasi yaitu dengan mengalokasikan dana pada berbagai alternatif investasi pada kombinasi saham pada portofolio. Pada skripsi ini, saham yang digunakan berupa saham dari perusahan sektor tambang yang terdapat 4 subsektor didalamnya yaitu subsektor Batubara, subsektor batu-batuan, subsektor logam dan mineral lainnya, dan subsektor minyak dan gas bumi. Dari setiap subs

Error fetching URL https://pta.trunojoyo.ac.id/c_search/byprod/9/64: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))
Error fetching URL https://pta.trunojoyo.ac.id/c_search/byprod/9/65: HTTPSConnectionPool(host='pta.trunojoyo.ac.id', port=443): Max retries exceeded with url: /c_search/byprod/9/65 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020875915310>: Failed to resolve 'pta.trunojoyo.ac.id' ([Errno 11001] getaddrinfo failed)"))
Error fetching URL https://pta.trunojoyo.ac.id/c_search/byprod/9/66: HTTPSConnectionPool(host='pta.trunojoyo.ac.id', port=443): Max retries exceeded with url: /c_search/byprod/9/66 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020875D28550>: Failed to resolve 'pta.trunojoyo.ac.id' ([Errno 11001] getaddrinfo failed)"))
Error fetching URL https://pta.trunojoyo.ac.id/c_search/byprod/


Scraping data for program: Teknik Informatika
--------------------------------------------------

Scraping data for program: Manajemen Informatika
--------------------------------------------------

Scraping data for program: Teknik Multimedia Dan Jaringan
--------------------------------------------------

Scraping data for program: Mekatronika
--------------------------------------------------

Scraping data for program: Teknik Elektro
--------------------------------------------------

Scraping data for program: Sistem Informasi
--------------------------------------------------

Scraping data for program: Teknik Mesin
--------------------------------------------------

Scraping data for program: Teknik Mekatronika
--------------------------------------------------

---
Scraping finished. Total data scraped: 315
Data saved to pta_teknik_complete.csv
---


,penulis,judul,pembimbing_pertama,pembimbing_kedua,abstrak_indonesia,abstrak_inggris,prodi
0,Siliwangi Fitra Rachmawanto S.T.,OPTIMASI PEMILIHAN PORTOFOLIO SAHAM PERUSAHAAN...,"Heri Awalul Ilhamsah S.T., M.T.","Retno Indriartiningtias S.T., M.T.",Portofolio adalah sekumpulan saham yang dimili...,Portofolio is a collection of stock owned by i...,Teknik Industri
1,AHMAD MAS'UD,PERANCANGAN TATA LETAK FASILITAS LANTAI PRODUK...,"SABARUDIN AKHMAD, S.T., M.T.","SUGENG PURWOKO, S.T., M.T.",PT. ABC merupakan perusahaan yang bergerak dib...,PT. ABC is a company engaged in the manufactur...,Teknik Industri
2,Yulianto Fauzanta,PERUMUSAN STRATEGI BISNIS UD. BUDI JAYA BANGKA...,"Fitri Agustina, S.T., M.T","Retno Indriartiningtias, S.T., M.T",Bangkalan merupakan salah satu kabupaten yang ...,Bangkalan is one of the districts that have th...,Teknik Industri
3,M Mundir Muhlisin,USULAN PERBAIKAN UTILITAS RESOURCES PADA LANTA...,Mu'alim ST MT,Sugeng Purwoko ST MT,Simulasi adalah duplikasi atau abstraksi dari ...,Simulation is a duplication or abstraction of ...,Teknik Industri
4,Muhibbin,Peningkatan Kepuasan Masyarakat Terhadap Pelay...,Rahmad Hidayat,Retno Indriartiningtias,Kepuasan adalah tingkat perasaan seseorang ter...,Satisfaction is feeling level of someone to se...,Teknik Industri
...,...,...,...,...,...,...,...
310,OKTO IMAM KHAMBALI,PENERAPAN METODE SIX SIGMA UNTUK PERBAIKAN KUA...,"Dr. Kukuh Winarso, S.Si.,M.T.","Issa Dyah Utami S.T., M.T., P.hD.",Industri pakan ternak yang terus meningkat dan...,Animal feed industry is increasing and competi...,Teknik Industri
311,Siti Maghfiroh,Penerapan Konsep Lean Manufacturing Dengan Val...,"Sugeng Purwoko S.T., M.T","Mu'alim S.T.,MT",Dalam proses pembuatan produk tentunya terdapa...,In the manufacturing process of course there i...,Teknik Industri
312,ABD.WAFI,Analisa Kelayakan Pemanfaatan Tenaga Surya Dan...,"MU'ALIM.S.T.,M.T","SAMSUL AMAR.S.T.,MSc",Dalam melakukan studi kelayakan dapat dilakuka...,In conducting the feasibility study can be don...,Teknik Industri
313,Agung Firdausi Ahsan,Perancangan Prototipe Lampu Senter Tenaga Pana...,"Mualim S.T, M.T","Dr. Rahmad Hidayat S.T, M.T",Indonesia adalah negara yang terdiri dari kepu...,Indonesia is a country that consists of small ...,Teknik Industri


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Konfigurasi
URL = "https://mulaab.github.io"  # Website target
FOLDER_NAME = "gambar"             # Folder untuk simpan gambar
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Buat folder jika belum ada
if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)
    print(f"Folder '{FOLDER_NAME}' berhasil dibuat.")

# Ambil halaman web
try:
    response = requests.get(URL, headers=HEADERS, timeout=10)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Gagal mengakses {URL}: {e}")
    exit()

# Parse HTML dengan BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Cari semua tag <img>
images = soup.find_all('img')
print(f"Ditemukan {len(images)} gambar di halaman {URL}.")

# List untuk menyimpan URL gambar
image_urls = []

for i, img in enumerate(images):
    # Ambil atribut src
    src = img.get('src')
    if not src:
        print(f"Gambar ke-{i+1} tidak memiliki atribut 'src', dilewati.")
        continue

    # Konversi URL relatif ke absolut
    img_url = urljoin(URL, src)
    image_urls.append(img_url)

    # Ambil nama file dari URL
    parsed_url = urlparse(img_url)
    filename = os.path.basename(parsed_url.path)

    # Validasi nama file
    if not filename or '.' not in filename:
        ext = '.jpg'
        filename = f"image_{i+1}{ext}"

    # Pastikan ekstensi file valid
    valid_exts = ('.png', '.jpg', '.jpeg', '.gif', '.webp', '.svg')
    if not filename.lower().endswith(valid_exts):
        filename += '.jpg'

    # Path lokal
    file_path = os.path.join(FOLDER_NAME, filename)

    # Download gambar
    try:
        img_response = requests.get(img_url, headers=HEADERS, timeout=10)
        img_response.raise_for_status()

        with open(file_path, 'wb') as f:
            f.write(img_response.content)

        print(f"Berhasil download: {filename} -> {file_path}")

    except Exception as e:
        print(f"Gagal download {img_url}: {e}")

# Ringkasan
print("\nCrawling selesai!")
print(f"Semua gambar disimpan di: ./{FOLDER_NAME}")
print(f"Total gambar diproses: {len(image_urls)}")

Folder 'gambar' berhasil dibuat.
Ditemukan 3 gambar di halaman https://mulaab.github.io.
Berhasil download: avatar_hubbf8cd69e817f868f48e6cd8dbaf10a0_30117_250x250_fill_q90_lanczos_center.jpg -> gambar\avatar_hubbf8cd69e817f868f48e6cd8dbaf10a0_30117_250x250_fill_q90_lanczos_center.jpg
Berhasil download: featured_huad4ca000f548177c81ea8fc52ac135a6_195982_150x0_resize_q90_lanczos.jpg -> gambar\featured_huad4ca000f548177c81ea8fc52ac135a6_195982_150x0_resize_q90_lanczos.jpg
Berhasil download: featured_hu3888f6a4f517b25077a8f129d425e8d6_206332_150x0_resize_lanczos_2.png -> gambar\featured_hu3888f6a4f517b25077a8f129d425e8d6_206332_150x0_resize_lanczos_2.png

Crawling selesai!
Semua gambar disimpan di: ./gambar
Total gambar diproses: 3


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

kategori_list = ["nasional", "bisnis", "tekno"]
base_rss = "https://rss.tempo.co"

results = []

for kategori in kategori_list:
    rss_url = f"{base_rss}/{kategori}"
    print(f"🔎 Ambil RSS: {rss_url}")
    resp = requests.get(rss_url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"❌ Gagal akses {rss_url}")
        continue

    soup = BeautifulSoup(resp.content, "xml")
    items = soup.find_all("item")

    for item in items:
        link = item.find("link").text
        judul = item.find("title").text.strip()

        try:
            berita_id = link.split("/")[-1].split("-")[0]
        except:
            berita_id = None

        # Ambil isi berita detail
        r = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        if r.status_code != 200:
            continue
        s = BeautifulSoup(r.text, "html.parser")

        isi_tag = s.find("div", class_="detail-konten")
        if isi_tag:
            for tag in isi_tag(["script", "style", "aside", "figure"]):
                tag.decompose()
            isi = isi_tag.get_text(" ", strip=True)
            isi = re.sub(r"Baca juga.*?(?=[A-Z])", "", isi)
        else:
            isi = None

        results.append([berita_id, judul, isi, kategori])
        print(f"✅ {judul} [{kategori}]")

        time.sleep(1)

# Simpan ke CSV
with open("berita_tempo.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_berita", "judul", "isi", "kategori"])
    writer.writerows(results)

print(f"🎉 Crawling selesai, total berita: {len(results)}, hasil disimpan ke berita_tempo.csv")

🔎 Ambil RSS: https://rss.tempo.co/nasional
✅ Daftar Menteri dan Wamen yang Diganti Prabowo [nasional]
✅ Cerita Djamari Chaniago Ditunjuk Prabowo Jadi Menko Polkam [nasional]
✅ Alasan Pemerintah Prioritaskan Pembahasan RUU Penyesuaian Pidana [nasional]
✅ Hasil Pertemuan Ojek Online-DPR, Skema Tarif Penumpang Diubah [nasional]
✅ BAZNAS RI Dukung Kemenko PM Wujudkan 1.001 Titik Pemberdayaan Berbasis Kawasan [nasional]
✅ RUU Perampasan Aset Diupayakan Tak Cuma Akomodasi Pidana Korupsi [nasional]
✅ Debat Sengit Komisi II DPR dan Baleg Berebut Bahas Revisi UU Pemilu [nasional]
✅ Sengketa Pilkada Ulang Barito Utara dan Papua Kandas di MK, Ini Alasannya [nasional]
✅ Respons Pemerintah dan Tentara soal MK Tolak Gugatan Uji Formil UU TNI [nasional]
✅ Rangkap Jabatan Menpora dan Ketum PSSI, Erick Tunggu Putusan FIFA [nasional]
✅ Hadiri Acara The Taste of Papua, Fatma Saifullah Yusuf Suka Kelezatan Papeda [nasional]
✅ Istana Akan Umumkan Anggota Tim Reformasi Kepolisian Pekan Ini [nasional]
✅ Ista

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

kategori_list = ["bola", "metro", "dunia"]
base_rss = "https://rss.tempo.co"

results = []

for kategori in kategori_list:
    rss_url = f"{base_rss}/{kategori}"
    print(f"🔎 Ambil RSS: {rss_url}")
    resp = requests.get(rss_url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"❌ Gagal akses {rss_url}")
        continue

    soup = BeautifulSoup(resp.content, "xml")
    items = soup.find_all("item")

    for item in items:
        link = item.find("link").text
        judul = item.find("title").text.strip()

        try:
            berita_id = link.split("/")[-1].split("-")[0]
        except:
            berita_id = None

        # Ambil isi berita detail
        r = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        if r.status_code != 200:
            continue
        s = BeautifulSoup(r.text, "html.parser")

        isi_tag = s.find("div", class_="detail-konten")
        if isi_tag:
            for tag in isi_tag(["script", "style", "aside", "figure"]):
                tag.decompose()
            isi = isi_tag.get_text(" ", strip=True)
            isi = re.sub(r"Baca juga.*?(?=[A-Z])", "", isi)
        else:
            isi = None

        results.append([berita_id, judul, isi, kategori])
        print(f"✅ {judul} [{kategori}]")

        time.sleep(1)

# Simpan ke CSV
with open("berita_tempo.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_berita", "judul", "isi", "kategori"])
    writer.writerows(results)

print(f"🎉 Crawling selesai, total berita: {len(results)}, hasil disimpan ke berita_tempo.csv")

🔎 Ambil RSS: https://rss.tempo.co/bola
✅ Cristiano Ronaldo Samai Rekor Gol Terbanyak di Kualifikasi Piala Dunia [bola]
✅ Alasan Pelatih Timnas U-23 Indonesia Enggan Bicara Performa Rafael Struick [bola]
✅ Hasil Kualifikasi Piala Dunia 2026: Kane, Mbappe, dan Ronaldo Persembahkan Kemenangan [bola]
✅ Catatan Evaluasi Pelatih Timnas U-23 Indonesia Usai Kalah Lawan Korea Selatan [bola]
✅ Klasemen Akhir Kualifikasi Piala Asia U-23 2026 dan Daftar Tim yang Lolos [bola]
✅ Usai Kalahkan Indonesia, Korea Selatan Targetkan Juarai Piala Asia U-23 2026 [bola]
✅ Timnas U-23 Indonesia Disingkirkan Korea. Vanenburg Bicara Soal Hati, Fisik, dan Taktik [bola]
✅ Kata Gerald Vanenburg setelah Gagal Bawa Timnas U-23 Indonesia Lolos ke Piala Asia [bola]
✅ Timnas U-23 Indonesia vs Korea Selatan 0-1, Garuda Muda Gagal ke Piala Asia [bola]
✅ Hasil Timnas U-23 Indonesia vs Korea Selatan, Babak 1 Skor 0-1 [bola]
✅ Susunan Pemain Timnas U-23 Indonesia vs Korea Selatan [bola]
✅ Pelatih Lebanon Jelaskan Alasan Mai

## Page & Link Keluar Berita

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import sys
import pandas as pd
def print_progress(current_item, total_items, category):
    """
    Menampilkan progress bar di konsol.
    """
    percent = (current_item / total_items) * 100 if total_items > 0 else 0
    bar_length = 20
    filled_length = int(bar_length * current_item // total_items)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{category}] Mengumpulkan tautan: {current_item}/{total_items} [{bar}] {percent:.2f}%')
    sys.stdout.flush()

def tempo_links_crawler():
    """
    Mengumpulkan semua tautan berita dari berbagai kategori Tempo.co
    dan menyimpannya dalam file CSV.
    """
    start_time = time.time()
    kategori_list = ["nasional", "bisnis", "tekno", "bola", "metro", "dunia"]
    base_rss = "https://rss.tempo.co"

    all_links_data = {
        "kategori": [],
        "link_keluar": []
    }

    total_entries = 0
    total_rss_items = len(kategori_list) * 15 # Asumsi setiap RSS feed memiliki 15 item

    print("=" * 80)
    print(f"{'No.':<4} | {'Kategori':<15} | {'Link Keluar'}")
    print("=" * 80)

    current_item_count = 0

    for kategori in kategori_list:
        rss_url = f"{base_rss}/{kategori}"

        try:
            resp = requests.get(rss_url, headers={"User-Agent": "Mozilla/5.0"})
            resp.raise_for_status() # Akan memunculkan error jika status code bukan 200

            soup = BeautifulSoup(resp.content, "xml")
            items = soup.find_all("item")

            if not items:
                continue

            for item in items:
                link = item.find("link").text

                all_links_data["kategori"].append(kategori)
                all_links_data["link_keluar"].append(link)

                total_entries += 1
                current_item_count += 1

                # Menampilkan data di konsol
                print(f"{total_entries:<4} | {kategori[:15]:<15} | {link}")

            # Menampilkan progress bar setelah setiap kategori
            print_progress(current_item_count, total_rss_items, kategori)

            time.sleep(1) # Menghindari request terlalu cepat

        except requests.exceptions.RequestException as e:
            print(f"❌ Gagal akses {rss_url}: {e}", file=sys.stderr)
            continue

    df = pd.DataFrame(all_links_data)
    df.to_csv("tempo_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n---")
    print(f"✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")
    print(f"Hasil disimpan ke tempo_links.csv")
    print("---")

    return df

# Jalankan proses crawling
tempo_links_crawler()

No.  | Kategori        | Link Keluar
1    | nasional        | https://nasional.tempo.co/read/2049854/menko-polkam-djamari-chaniago-evaluasi-desk-penindakan-era-bg
2    | nasional        | https://nasional.tempo.co/read/2049848/daftar-menteri-dan-wamen-yang-diganti-prabowo
3    | nasional        | https://nasional.tempo.co/read/2049846/cerita-djamari-chaniago-ditunjuk-prabowo-jadi-menko-polkam
4    | nasional        | https://nasional.tempo.co/read/2049812/alasan-pemerintah-prioritaskan-pembahasan-ruu-penyesuaian-pidana
5    | nasional        | https://nasional.tempo.co/read/2049819/hasil-pertemuan-ojek-online-dpr-skema-tarif-penumpang-diubah
6    | nasional        | https://nasional.tempo.co/read/2049803/baznas-ri-dukung-kemenko-pm-wujudkan-1-001-titik-pemberdayaan-berbasis-kawasan
7    | nasional        | https://nasional.tempo.co/read/2049801/ruu-perampasan-aset-diupayakan-tak-cuma-akomodasi-pidana-korupsi
8    | nasional        | https://nasional.tempo.co/read/2049798/debat-sengit-k

,kategori,link_keluar
0,nasional,https://nasional.tempo.co/read/2049854/menko-p...
1,nasional,https://nasional.tempo.co/read/2049848/daftar-...
2,nasional,https://nasional.tempo.co/read/2049846/cerita-...
3,nasional,https://nasional.tempo.co/read/2049812/alasan-...
4,nasional,https://nasional.tempo.co/read/2049819/hasil-p...
...,...,...
295,dunia,https://dunia.tempo.co/read/2048954/negara-ara...
296,dunia,https://dunia.tempo.co/read/2048951/siapa-geng...
297,dunia,https://dunia.tempo.co/read/2048948/top-3-duni...
298,dunia,https://dunia.tempo.co/read/2048895/menteri-ma...


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import sys
import pandas as pd

In [ ]:
def scrape_artikel_tempo(url, kategori):
    """
    Fungsi ini mengunjungi satu URL artikel Tempo, mengambil judul dan isinya.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status() # Akan error jika status code bukan 200

        soup = BeautifulSoup(response.text, 'lxml')

        # 1. Ambil Judul Berita dari tag <title>
        judul_berita = soup.find('title').get_text(strip=True) if soup.find('title') else "Judul Tidak Ditemukan"

        # 2. Ambil Isi Berita dari <div id="content-wrapper">
        wrapper = soup.find('div', id='content-wrapper')
        if wrapper:
            # Ambil semua teks dari paragraf <p> di dalam wrapper
            paragraf = wrapper.find_all('p')
            # Gabungkan semua paragraf menjadi satu teks utuh
            isi_berita = "\n\n".join([p.get_text(strip=True) for p in paragraf])
        else:
            isi_berita = "Konten Tidak Ditemukan"

        # Kembalikan hasil dalam bentuk dictionary
        return {
            'judul_berita': judul_berita,
            'isi': isi_berita,
            'kategori': kategori
        }

    except requests.exceptions.RequestException as e:
        print(f"  -> Terjadi error koneksi: {e}")
        return None
    except Exception as e:
        print(f"  -> Terjadi error saat memproses {url}: {e}")
        return None

In [ ]:
import pandas as pd
# --- PENGATURAN ---
FILE_INPUT = 'tempo_links.csv'
FILE_OUTPUT = 'berita_tempo.csv'

print(f"🚀 Memulai proses scraping dari file '{FILE_INPUT}'...")

# --- MEMBACA FILE INPUT ---
try:
    df_links = pd.read_csv(FILE_INPUT)
    print(f"✅ Berhasil memuat {len(df_links)} link untuk di-scrape.")
except FileNotFoundError:
    print(f"❌ File '{FILE_INPUT}' tidak ditemukan! Pastikan file berada di folder yang sama.")
    df_links = pd.DataFrame() # Buat dataframe kosong agar tidak error

# --- PROSES SCRAPING ---
if not df_links.empty:
    semua_data = []

    # Looping melalui setiap baris di DataFrame
    for index, row in df_links.iterrows():
        # --- PERBAIKAN DI SINI ---
        link = row['link_keluar']  # Menggunakan nama kolom yang benar
        kategori = row['kategori']
        # -------------------------

        print(f"Mengambil data ke-{index + 1}: {link[:70]}...") # Tampilkan sebagian link

        hasil = scrape_artikel_tempo(link, kategori)

        if hasil:
            hasil['id'] = index + 1
            semua_data.append(hasil)

        time.sleep(1) # Jeda 1 detik antar request

    # --- MENYIMPAN HASIL KE CSV BARU ---
    if semua_data:
        print(f"\n✅ Proses scraping selesai. Total {len(semua_data)} artikel berhasil diambil.")

        df_hasil = pd.DataFrame(semua_data)

        # Atur urutan kolom sesuai permintaan: id, judul berita, isi, kategori
        df_hasil = df_hasil[['id', 'judul_berita', 'isi', 'kategori']]

        df_hasil.to_csv(FILE_OUTPUT, index=False, encoding='utf-8-sig')

        print(f"✨ Data berhasil disimpan di file: '{FILE_OUTPUT}'")

        print("\n--- 5 Baris Pertama Hasil Scraping ---")
        print(df_hasil.head())

    else:
        print("\nTidak ada data yang berhasil di-scrape.")

🚀 Memulai proses scraping dari file 'tempo_links.csv'...
✅ Berhasil memuat 300 link untuk di-scrape.
Mengambil data ke-1: https://nasional.tempo.co/read/2049854/menko-polkam-djamari-chaniago-e...
  -> Terjadi error koneksi: HTTPSConnectionPool(host='nasional.tempo.co', port=443): Max retries exceeded with url: /read/2049854/menko-polkam-djamari-chaniago-evaluasi-desk-penindakan-era-bg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020860D93390>: Failed to resolve 'nasional.tempo.co' ([Errno 11001] getaddrinfo failed)"))
Mengambil data ke-2: https://nasional.tempo.co/read/2049848/daftar-menteri-dan-wamen-yang-d...
  -> Terjadi error koneksi: HTTPSConnectionPool(host='nasional.tempo.co', port=443): Max retries exceeded with url: /read/2049848/daftar-menteri-dan-wamen-yang-diganti-prabowo (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020860D92AD0>: Failed to resolve 'nasional.tempo.co' ([Errno 11001] getaddrinfo fa

: 